In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split, cross_val_score
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import pycountry
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kick-off/sample_submission.csv
/kaggle/input/kick-off/FIFA_test_player_data.csv
/kaggle/input/kick-off/FIFA_train.csv
/kaggle/input/kick-off/FIFA_test.csv


In [2]:
data=pd.read_csv('../input/kick-off/FIFA_train.csv')
test_data=pd.read_csv('../input/kick-off/FIFA_test_player_data.csv')
test_data

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Neuer,31,1986-03-27,193,92,Germany,FC Bayern München,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,155862,https://sofifa.com/player/155862/sergio-ramos-...,Sergio Ramos,Sergio Ramos García,31,1986-03-30,183,75,Spain,Real Madrid,...,82+3,83+3,83+3,83+3,82+3,84+3,87+3,87+3,87+3,84+3
2,182521,https://sofifa.com/player/182521/toni-kroos/18...,T. Kroos,Toni Kroos,27,1990-01-04,182,78,Germany,Real Madrid,...,79+3,82+3,82+3,82+3,79+3,76+3,73+3,73+3,73+3,76+3
3,138956,https://sofifa.com/player/138956/giorgio-chiel...,G. Chiellini,Giorgio Chiellini,32,1984-08-14,187,85,Italy,Juventus,...,75+3,77+3,77+3,77+3,75+3,79+3,87+3,87+3,87+3,79+3
4,9014,https://sofifa.com/player/9014/arjen-robben/18...,A. Robben,Arjen Robben,33,1984-01-23,180,80,Netherlands,FC Bayern München,...,63+3,59+3,59+3,59+3,63+3,58+3,47+3,47+3,47+3,58+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16087,246694,https://sofifa.com/player/246694/sean-callan/2...,S. Callan,Sean Callan,19,1999-12-14,182,80,Republic of Ireland,Shamrock Rovers,...,43+2,40+2,40+2,40+2,43+2,45+2,47+2,47+2,47+2,45+2
16088,251124,https://sofifa.com/player/251124/francisco-nev...,F. Nevarez,Francisco Nevarez,18,2000-12-03,178,69,Mexico,FC Juárez,...,46+2,41+2,41+2,41+2,46+2,47+2,46+2,46+2,46+2,47+2
16089,245029,https://sofifa.com/player/245029/peng-wang/20/...,Wang Peng,王鹏,21,1997-11-16,175,70,China PR,Guangzhou R&F FC,...,43+2,43+2,43+2,43+2,43+2,43+2,40+2,40+2,40+2,43+2
16090,252181,https://sofifa.com/player/252181/jiahui-huang/...,Huang Jiahui,Jiahui Huang,18,2000-10-07,183,74,China PR,Dalian YiFang FC,...,44+2,40+2,40+2,40+2,44+2,46+2,46+2,46+2,46+2,46+2


In [3]:
data['train']=1
test_data['train']=0

In [4]:
combined=pd.concat([data,test_data])

In [5]:
import category_encoders as ce
ce_ordinal = ce.OrdinalEncoder(cols=['nationality','club','preferred_foot','work_rate','body_type','real_face','team_position','nation_position','player_positions'])
combined=ce_ordinal.fit_transform(combined)

In [6]:
data=combined[combined['train']==1]
test=combined[combined['train']==0]

In [7]:
data.columns

Index(['sofifa_id', 'player_url', 'short_name', 'long_name', 'age', 'dob',
       'height_cm', 'weight_kg', 'nationality', 'club',
       ...
       'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'train'],
      dtype='object', length=104)

In [8]:
data=data.drop(['player_url','short_name','long_name','dob','player_tags','player_traits'],axis='columns')
data=data.drop(['team_jersey_number','loaned_from','joined','contract_valid_until','nation_jersey_number','train'],axis='columns')
test=test.drop(['player_url','short_name','long_name','dob','player_tags','player_traits'],axis='columns')
test=test.drop(['team_jersey_number','loaned_from','joined','contract_valid_until','nation_jersey_number','train'],axis='columns')

In [9]:
test=test.drop(['value'],axis='columns')
test

,sofifa_id,age,height_cm,weight_kg,nationality,club,overall,potential,player_positions,preferred_foot,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,167495,31,193,92,9,101,92,92,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,155862,31,183,75,7,464,90,90,8,1,...,82+3,83+3,83+3,83+3,82+3,84+3,87+3,87+3,87+3,84+3
2,182521,27,182,78,9,464,90,90,1,1,...,79+3,82+3,82+3,82+3,79+3,76+3,73+3,73+3,73+3,76+3
3,138956,32,187,85,3,657,89,89,8,2,...,75+3,77+3,77+3,77+3,75+3,79+3,87+3,87+3,87+3,79+3
4,9014,33,180,80,38,101,88,88,66,2,...,63+3,59+3,59+3,59+3,63+3,58+3,47+3,47+3,47+3,58+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16087,246694,19,182,80,14,462,48,61,43,1,...,43+2,40+2,40+2,40+2,43+2,45+2,47+2,47+2,47+2,45+2
16088,251124,18,178,69,47,770,48,61,27,1,...,46+2,41+2,41+2,41+2,46+2,47+2,46+2,46+2,46+2,47+2
16089,245029,21,175,70,12,788,48,60,10,1,...,43+2,43+2,43+2,43+2,43+2,43+2,40+2,40+2,40+2,43+2
16090,252181,18,183,74,12,689,48,60,8,1,...,44+2,40+2,40+2,40+2,44+2,46+2,46+2,46+2,46+2,46+2


In [10]:
data

,sofifa_id,age,height_cm,weight_kg,nationality,club,overall,potential,value,player_positions,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,216393,21,176,72,1,1,79,89,20000000.0,1,...,71+2,74+2,74+2,74+2,71+2,69+2,69+2,69+2,69+2,69+2
1,187878,28,183,79,2,2,71,71,2400000.0,2,...,71+2,71+2,71+2,71+2,71+2,70+2,68+2,68+2,68+2,70+2
2,178628,29,172,67,3,3,73,73,4000000.0,3,...,60+2,60+2,60+2,60+2,60+2,58+2,56+2,56+2,56+2,58+2
3,184341,31,183,75,3,4,69,69,950000.0,4,...,43+2,46+2,46+2,46+2,43+2,41+2,44+2,44+2,44+2,41+2
4,231933,25,185,75,4,5,58,64,0.0,5,...,56+1,57+1,57+1,57+1,56+1,57+1,58+1,58+1,58+1,57+1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37543,242308,28,177,75,96,291,62,62,300000.0,48,...,51+2,50+2,50+2,50+2,51+2,49+2,42+2,42+2,42+2,49+2
37544,186259,29,181,70,40,94,66,66,525000.0,55,...,64+2,63+2,63+2,63+2,64+2,64+2,65+2,65+2,65+2,64+2
37545,218623,24,183,84,24,56,73,78,4500000.0,20,...,70+1,73+1,73+1,73+1,70+1,70+1,73+1,73+1,73+1,70+1
37546,244469,28,175,71,46,24,59,60,140000.0,20,...,57+2,57+2,57+2,57+2,57+2,57+2,57+2,57+2,57+2,57+2


In [11]:
x=data.columns[32:92]

k=0
i=0
j=32
while k<60:
    d=data[x[k]].squeeze()
    d=d.str.replace('-', '+')
    new = d.str.split("+", n = 1, expand = True)
    data['cb1']= new[0]
    data["cb2"]= new[1]
    data[data.columns[j]]=data["cb1"].astype(float) + data["cb2"].astype(float)
    data.drop(columns =["cb1"], inplace = True)
    data.drop(columns =["cb2"], inplace = True)
    i=i+2
    j=j+1
    k=k+1
data

,sofifa_id,age,height_cm,weight_kg,nationality,club,overall,potential,value,player_positions,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,216393,21,176,72,1,1,79,89,20000000.0,1,...,73.0,76.0,76.0,76.0,73.0,71.0,71.0,71.0,71.0,71.0
1,187878,28,183,79,2,2,71,71,2400000.0,2,...,73.0,73.0,73.0,73.0,73.0,72.0,70.0,70.0,70.0,72.0
2,178628,29,172,67,3,3,73,73,4000000.0,3,...,62.0,62.0,62.0,62.0,62.0,60.0,58.0,58.0,58.0,60.0
3,184341,31,183,75,3,4,69,69,950000.0,4,...,45.0,48.0,48.0,48.0,45.0,43.0,46.0,46.0,46.0,43.0
4,231933,25,185,75,4,5,58,64,0.0,5,...,57.0,58.0,58.0,58.0,57.0,58.0,59.0,59.0,59.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37543,242308,28,177,75,96,291,62,62,300000.0,48,...,53.0,52.0,52.0,52.0,53.0,51.0,44.0,44.0,44.0,51.0
37544,186259,29,181,70,40,94,66,66,525000.0,55,...,66.0,65.0,65.0,65.0,66.0,66.0,67.0,67.0,67.0,66.0
37545,218623,24,183,84,24,56,73,78,4500000.0,20,...,71.0,74.0,74.0,74.0,71.0,71.0,74.0,74.0,74.0,71.0
37546,244469,28,175,71,46,24,59,60,140000.0,20,...,59.0,59.0,59.0,59.0,59.0,59.0,59.0,59.0,59.0,59.0


In [12]:
d=data.dtypes
d[0:30]

sofifa_id                     int64
age                           int64
height_cm                     int64
weight_kg                     int64
nationality                   int64
club                          int64
overall                       int64
potential                     int64
value                       float64
player_positions              int64
preferred_foot                int64
international_reputation      int64
weak_foot                     int64
skill_moves                   int64
work_rate                     int64
body_type                     int64
real_face                     int64
release_clause_eur          float64
team_position                 int64
nation_position               int64
pace                        float64
shooting                    float64
passing                     float64
dribbling                   float64
defending                   float64
physic                      float64
gk_diving                   float64
gk_handling                 

In [13]:
x=test.columns[31:93]

k=0
i=0
j=31
while k<60:
    d=test[x[k]].squeeze()
    d=d.str.replace('-', '+')
    new = d.str.split("+", n = 1, expand = True)
    test['cb1']= new[0]
    test["cb2"]= new[1]
    test[test.columns[j]]=test["cb1"].astype(float) + test["cb2"].astype(float)
    test.drop(columns =["cb1"], inplace = True)
    test.drop(columns =["cb2"], inplace = True)
    i=i+2
    j=j+1
    k=k+1
test

,sofifa_id,age,height_cm,weight_kg,nationality,club,overall,potential,player_positions,preferred_foot,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,167495,31,193,92,9,101,92,92,7,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,155862,31,183,75,7,464,90,90,8,1,...,85.0,86.0,86.0,86.0,85.0,87.0,90.0,90.0,90.0,87.0
2,182521,27,182,78,9,464,90,90,1,1,...,82.0,85.0,85.0,85.0,82.0,79.0,76.0,76.0,76.0,79.0
3,138956,32,187,85,3,657,89,89,8,2,...,78.0,80.0,80.0,80.0,78.0,82.0,90.0,90.0,90.0,82.0
4,9014,33,180,80,38,101,88,88,66,2,...,66.0,62.0,62.0,62.0,66.0,61.0,50.0,50.0,50.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16087,246694,19,182,80,14,462,48,61,43,1,...,45.0,42.0,42.0,42.0,45.0,47.0,49.0,49.0,49.0,47.0
16088,251124,18,178,69,47,770,48,61,27,1,...,48.0,43.0,43.0,43.0,48.0,49.0,48.0,48.0,48.0,49.0
16089,245029,21,175,70,12,788,48,60,10,1,...,45.0,45.0,45.0,45.0,45.0,45.0,42.0,42.0,42.0,45.0
16090,252181,18,183,74,12,689,48,60,8,1,...,46.0,42.0,42.0,42.0,46.0,48.0,48.0,48.0,48.0,48.0


In [14]:
data.columns

Index(['sofifa_id', 'age', 'height_cm', 'weight_kg', 'nationality', 'club',
       'overall', 'potential', 'value', 'player_positions', 'preferred_foot',
       'international_reputation', 'weak_foot', 'skill_moves', 'work_rate',
       'body_type', 'real_face', 'release_clause_eur', 'team_position',
       'nation_position', 'pace', 'shooting', 'passing', 'dribbling',
       'defending', 'physic', 'gk_diving', 'gk_handling', 'gk_kicking',
       'gk_reflexes', 'gk_speed', 'gk_positioning', 'attacking_crossing',
       'attacking_finishing', 'attacking_heading_accuracy',
       'attacking_short_passing', 'attacking_volleys', 'skill_dribbling',
       'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
       'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed',
       'movement_agility', 'movement_reactions', 'movement_balance',
       'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
       'power_long_shots', 'mentality_aggression', 'ment

In [15]:
test.columns

Index(['sofifa_id', 'age', 'height_cm', 'weight_kg', 'nationality', 'club',
       'overall', 'potential', 'player_positions', 'preferred_foot',
       'international_reputation', 'weak_foot', 'skill_moves', 'work_rate',
       'body_type', 'real_face', 'release_clause_eur', 'team_position',
       'nation_position', 'pace', 'shooting', 'passing', 'dribbling',
       'defending', 'physic', 'gk_diving', 'gk_handling', 'gk_kicking',
       'gk_reflexes', 'gk_speed', 'gk_positioning', 'attacking_crossing',
       'attacking_finishing', 'attacking_heading_accuracy',
       'attacking_short_passing', 'attacking_volleys', 'skill_dribbling',
       'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
       'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed',
       'movement_agility', 'movement_reactions', 'movement_balance',
       'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
       'power_long_shots', 'mentality_aggression', 'mentality_int

In [16]:
for i in data.columns[data.isnull().any(axis=0)]:     #---Applying Only on variables with NaN values
    data[i].fillna(data[i].mean(),inplace=True)
    
for i in test.columns[test.isnull().any(axis=0)]:     #---Applying Only on variables with NaN values
    test[i].fillna(test[i].mean(),inplace=True)

In [17]:
data.duplicated().sum()

0

In [18]:
data.isnull().sum()

sofifa_id      0
age            0
height_cm      0
weight_kg      0
nationality    0
              ..
lb             0
lcb            0
cb             0
rcb            0
rb             0
Length: 92, dtype: int64

In [19]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16092 entries, 0 to 16091
Data columns (total 91 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   sofifa_id                   16092 non-null  int64  
 1   age                         16092 non-null  int64  
 2   height_cm                   16092 non-null  int64  
 3   weight_kg                   16092 non-null  int64  
 4   nationality                 16092 non-null  int64  
 5   club                        16092 non-null  int64  
 6   overall                     16092 non-null  int64  
 7   potential                   16092 non-null  int64  
 8   player_positions            16092 non-null  int64  
 9   preferred_foot              16092 non-null  int64  
 10  international_reputation    16092 non-null  int64  
 11  weak_foot                   16092 non-null  int64  
 12  skill_moves                 16092 non-null  int64  
 13  work_rate                   160

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37548 entries, 0 to 37547
Data columns (total 92 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   sofifa_id                   37548 non-null  int64  
 1   age                         37548 non-null  int64  
 2   height_cm                   37548 non-null  int64  
 3   weight_kg                   37548 non-null  int64  
 4   nationality                 37548 non-null  int64  
 5   club                        37548 non-null  int64  
 6   overall                     37548 non-null  int64  
 7   potential                   37548 non-null  int64  
 8   value                       37548 non-null  float64
 9   player_positions            37548 non-null  int64  
 10  preferred_foot              37548 non-null  int64  
 11  international_reputation    37548 non-null  int64  
 12  weak_foot                   37548 non-null  int64  
 13  skill_moves                 375

In [21]:
X = data.copy()
y = X.pop('value')

In [22]:
data

,sofifa_id,age,height_cm,weight_kg,nationality,club,overall,potential,value,player_positions,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,216393,21,176,72,1,1,79,89,20000000.0,1,...,73.0,76.0,76.0,76.0,73.0,71.0,71.0,71.0,71.0,71.0
1,187878,28,183,79,2,2,71,71,2400000.0,2,...,73.0,73.0,73.0,73.0,73.0,72.0,70.0,70.0,70.0,72.0
2,178628,29,172,67,3,3,73,73,4000000.0,3,...,62.0,62.0,62.0,62.0,62.0,60.0,58.0,58.0,58.0,60.0
3,184341,31,183,75,3,4,69,69,950000.0,4,...,45.0,48.0,48.0,48.0,45.0,43.0,46.0,46.0,46.0,43.0
4,231933,25,185,75,4,5,58,64,0.0,5,...,57.0,58.0,58.0,58.0,57.0,58.0,59.0,59.0,59.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37543,242308,28,177,75,96,291,62,62,300000.0,48,...,53.0,52.0,52.0,52.0,53.0,51.0,44.0,44.0,44.0,51.0
37544,186259,29,181,70,40,94,66,66,525000.0,55,...,66.0,65.0,65.0,65.0,66.0,66.0,67.0,67.0,67.0,66.0
37545,218623,24,183,84,24,56,73,78,4500000.0,20,...,71.0,74.0,74.0,74.0,71.0,71.0,74.0,74.0,74.0,71.0
37546,244469,28,175,71,46,24,59,60,140000.0,20,...,59.0,59.0,59.0,59.0,59.0,59.0,59.0,59.0,59.0,59.0


In [23]:
y =data['value']
X =data.drop(['value'],axis='columns')
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [24]:
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
model = RandomForestClassifier(random_state=1)
model.fit(train_X, train_y)
pred_y = model.predict(val_X)
accuracy = metrics.accuracy_score(pred_y,val_y) * 100
accuracy

35.09108341323107

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
lr = LinearRegression()
lr.fit(train_X,train_y)
y_pred = lr.predict(val_X)
r2_score(val_y, y_pred)

0.9560857542539872

In [26]:
my_model = XGBRegressor(n_estimators=5000, learning_rate=0.01)
my_model.fit(train_X, train_y, 
             early_stopping_rounds=5, 
             eval_set=[(val_X, val_y)], 
             verbose=False)
y_pred = my_model.predict(val_X)
r2_score(val_y, y_pred)

0.9925463757114061

In [27]:
pred=my_model.predict(test)
pred

array([5.5321056e+07, 5.2064444e+07, 7.6632936e+07, ..., 7.1069008e+04,
       6.2278367e+04, 6.3571609e+04], dtype=float32)

In [28]:
fitest=pd.read_csv('../input/kick-off/FIFA_test.csv')

In [29]:
for i in range(len(fitest['player_ids'])):
    fitest['player_ids'][i]=fitest['player_ids'][i].split(",")
    for j in range(len(fitest['player_ids'][i])):
        fitest['player_ids'][i][j]=fitest['player_ids'][i][j].replace("[","")
        fitest['player_ids'][i][j]=fitest['player_ids'][i][j].replace("]","")
        fitest['player_ids'][i][j]=int(fitest['player_ids'][i][j])

In [30]:
data['value'].astype(float)
data

,sofifa_id,age,height_cm,weight_kg,nationality,club,overall,potential,value,player_positions,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,216393,21,176,72,1,1,79,89,20000000.0,1,...,73.0,76.0,76.0,76.0,73.0,71.0,71.0,71.0,71.0,71.0
1,187878,28,183,79,2,2,71,71,2400000.0,2,...,73.0,73.0,73.0,73.0,73.0,72.0,70.0,70.0,70.0,72.0
2,178628,29,172,67,3,3,73,73,4000000.0,3,...,62.0,62.0,62.0,62.0,62.0,60.0,58.0,58.0,58.0,60.0
3,184341,31,183,75,3,4,69,69,950000.0,4,...,45.0,48.0,48.0,48.0,45.0,43.0,46.0,46.0,46.0,43.0
4,231933,25,185,75,4,5,58,64,0.0,5,...,57.0,58.0,58.0,58.0,57.0,58.0,59.0,59.0,59.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37543,242308,28,177,75,96,291,62,62,300000.0,48,...,53.0,52.0,52.0,52.0,53.0,51.0,44.0,44.0,44.0,51.0
37544,186259,29,181,70,40,94,66,66,525000.0,55,...,66.0,65.0,65.0,65.0,66.0,66.0,67.0,67.0,67.0,66.0
37545,218623,24,183,84,24,56,73,78,4500000.0,20,...,71.0,74.0,74.0,74.0,71.0,71.0,74.0,74.0,74.0,71.0
37546,244469,28,175,71,46,24,59,60,140000.0,20,...,59.0,59.0,59.0,59.0,59.0,59.0,59.0,59.0,59.0,59.0


In [31]:
sub=pd.DataFrame()
sub['id']=test['sofifa_id'].astype(int)
sub['pred']=pred

sub2=pd.DataFrame()
sub2['id']=X['sofifa_id'].astype(int)
sub2['pred']=y
sub=pd.concat([sub,sub2])
sub = sub.drop_duplicates(keep="first")
sub.duplicated().sum()

0

In [32]:
sub=sub.reset_index(drop=True)
sub

,id,pred
0,167495,55321056.0
1,155862,52064444.0
2,182521,76632936.0
3,138956,33670028.0
4,9014,35801176.0
...,...,...
52336,242308,300000.0
52337,186259,525000.0
52338,218623,4500000.0
52339,244469,140000.0


In [33]:
sub['pred'][0]

55321056.0

In [34]:
most_valuable_player=[]
l=list(sub['id'])
d=list(sub['pred'])
for i in fitest['player_ids']:
    a=[]
    for j in i:
        c=l.index(j)
        a.append(sub['pred'][c])
    most_valuable_player.append(sub['id'][d.index(max(a))])

In [35]:
len(most_valuable_player)

1462

In [36]:
fitest['most_valued_player_id']=most_valuable_player

In [37]:
fitest

,player_ids,most_valued_player_id
0,"[238577, 189730, 212616, 209729, 246565, 23309...",196143
1,"[227675, 177793, 199511, 204387, 199964, 24472...",176619
2,"[199522, 225810, 227740, 243139, 229771, 18347...",180403
3,"[223953, 222399, 231307, 196143, 212814, 24479...",212814
4,"[53110, 229137, 213887, 179891, 228595, 211736...",228595
...,...,...
1457,"[239090, 227770, 211245, 233085, 232017, 22242...",146758
1458,"[220994, 231733, 205340, 240738, 241743, 23940...",220414
1459,"[228419, 224344, 222599, 210690, 209440, 23113...",152908
1460,"[205496, 223306, 210007, 232110, 192641, 23776...",193080


In [38]:
fitest.to_csv("submission.csv")